In [85]:
import pandas as pd

In [86]:
df = pd.read_csv("whole_quarter_cities_venues.csv")
df.drop_duplicates(inplace=True)

Attraction name = artist(s), format of artist|artist, split so that matching can happen
expand df such that each row contains only 1 artist, event id + artist will be the primary key

In [87]:
exploded_series = df['ATTRACTION_NAME'].str.split('|').explode()
df_expanded = df.reindex(exploded_series.index)
df_expanded['individual_artist'] = exploded_series.str.lower()

In [88]:
spotify_df = pd.read_csv("spotify_artist_data_2023.csv")
spotify_df.columns

Index(['id', 'name', 'artist_popularity', 'artist_genres', 'followers',
       'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5',
       'genre_6'],
      dtype='object')

popularity score assigned to concert is popularity of most popular artist

In [89]:
spotify_df["lower_name"] = spotify_df["name"].str.lower()
merged = df_expanded.merge(spotify_df, how="left", left_on="individual_artist", right_on="lower_name")
just_popular = merged[["EVENT_ID", "artist_popularity"]].groupby("EVENT_ID").max()
df_with_popularity = merged.merge(
    just_popular, 
    how="inner", 
    left_on=["EVENT_ID", "artist_popularity"],
    right_on=["EVENT_ID", "artist_popularity"]
)

Get the cities with missing population

In [90]:
full_cities_df = pd.read_csv("whole_quarter+cities2.csv")[["EVENT_ID", "population"]]
popularity_and_cities = df_with_popularity.merge(full_cities_df, how="inner", left_on="EVENT_ID", right_on="EVENT_ID")
popularity_and_cities["population"] = popularity_and_cities["population_y"]

missing artists fill in

In [91]:
missing_artists_df = pd.read_csv("output.csv").drop_duplicates()
missing_artists_df["lower_artist_name"] = missing_artists_df["artist_name"].str.lower()
have_popularity = missing_artists_df[missing_artists_df["scraped_popularity"] != -1]
final = popularity_and_cities.merge(
    have_popularity, 
    how="inner", 
    left_on="individual_artist", 
    right_on="lower_artist_name"
)

In [92]:
final.shape

(3668, 53)

In [93]:
final["artist_popularity"] = final["artist_popularity"].fillna(final["scraped_popularity"])
final.drop_duplicates(inplace=True)

In [94]:
final.to_csv("final.csv")